In [30]:
import pandas as pd
import json
from word2number import w2n
import pymongo
import numpy as np

In [13]:
config_file = open("../config.cfg")
config = json.load(config_file)
client = pymongo.MongoClient(config["mongodb_uri"])
db = client[config["database"]]
collection = db["rs_all_members"]

In [49]:
data = pd.read_excel("Dat_file.xlsx")

In [70]:
final_data = []

In [10]:
data.columns

Index(['Member Name', 'Gender', 'Email ID', 'Delhi Address',
       'Permanent Address', 'Whether Minister', 'Term Details',
       'Number of Terms', 'State Name', 'Party Name', 'Father Name',
       'Mother Name', 'Date of Birth', 'Place of Birth', 'Marital Status',
       'Date of Marriage', 'Spouse Name', 'No. of Sons', 'No. of Daughters',
       'Freedom Fighter', 'Educational Qualification(s)',
       'Other Profession(s)', 'Previous Membership', 'Position(s) Held',
       'Book(s) Published', 'Activity(s)', 'Hobbies', 'Country(s) Visited',
       'Essential Information'],
      dtype='object')

In [50]:
data.isnull().any()

Member Name                     False
Gender                          False
Email ID                         True
Delhi Address                    True
Permanent Address                True
Whether Minister                False
Term Details                     True
Number of Terms                 False
State Name                      False
Party Name                      False
Father Name                      True
Mother Name                      True
Date of Birth                    True
Place of Birth                   True
Marital Status                   True
Date of Marriage                 True
Spouse Name                      True
No. of Sons                      True
No. of Daughters                 True
Freedom Fighter                 False
Educational Qualification(s)     True
Other Profession(s)              True
Previous Membership              True
Position(s) Held                 True
Book(s) Published                True
Activity(s)                      True
Hobbies     

In [51]:
data = data.fillna("Not Available")

In [76]:
for i,row in data.iterrows():
    new_row = {}
    new_row["_id"] = str(1000+i)
    new_row["name"] = row["Member Name"]
    new_row["constituency"] = "Not Applicable"
    new_row["state"] = row["State Name"]
    new_row["party"] = row["Party Name"]
    new_row["email"] = row["Email ID"].split(";")
    new_row["dob"] = row["Date of Birth"]
    new_row["birth_place"] = row["Place of Birth"]
    new_row["marital_status"] = row["Marital Status"]
    if row ["Term Details"] == "Not Available":
        new_row["sessions"] = []
    else:
        sessions = [_.strip() for _ in row["Term Details"].strip(",").split(",")]
        sessions_data = []
        for session in sessions:
            sessions_data.append(
                {
                    "start_date" : session.split("-")[0].strip(),
                    "end_date" : session.split("-")[1].strip()
                })
            
        new_row["sessions"] = sessions_data
    if row["No. of Sons"] == "Not Available":
        new_row["sons"] = 0
    elif type(row["No. of Sons"]) == int:
        new_row["sons"] = row["No. of Sons"]
    else:
        new_row["sons"] = w2n.word_to_num(row["No. of Sons"])
    if row["No. of Daughters"] == "Not Available":
        new_row["daughters"] = 0
    elif type(row["No. of Daughters"]) == int:
        new_row["daughters"] = row["No. of Daughters"]
    else:
        new_row["daughters"] = w2n.word_to_num(row["No. of Daughters"])
    new_row["education"] = row["Educational Qualification(s)"]
    new_row["profession"] = row["Other Profession(s)"]
    final_data.append(new_row)

In [79]:
collection.insert_many(final_data)